In [7]:
import dynamicrouting_summary as dr
import pandas as pd
import oursin as urchin
import numpy as np
import pyarrow.parquet as pq
import npc_lims
import pathlib
import npc_session
import s3fs
import sqlite3
from sqlalchemy import create_engine

In [8]:
VERSION = 'v0.0.195'

In [9]:
 # allow integers >8 bytes to be stored in sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
# ------------------------------------------------------------------------------------
DB_PATH = pathlib.Path('//allen/programs/mindscope/workgroups/dynamicrouting/dynamic_gating_insertions/dr_master.db')
# with contextlib.suppress(OSError):
#     DB_PATH.unlink()
sqlite3.connect(DB_PATH).close()
DB = f"sqlite:///{DB_PATH}"
ENGINE = create_engine(DB, echo=False)

df_ccf = pd.read_sql('channel_ccf_coords', con=ENGINE)[['session', 'MID', 'Probe', 'Implant', 'Hole']]
df_ccf

,session,MID,Probe,Implant,Hole
0,1167400342,599657,A,41,None
1,1167400342,599657,B,41,None
2,1167400342,599657,C,41,None
3,1167400342,599657,D,41,None
4,1167400342,599657,E,41,None
...,...,...,...,...,...
1207,686176_2023-12-07,686176,B,2002,F1
1208,686176_2023-12-07,686176,C,2002,F3
1209,686176_2023-12-07,686176,D,2002,D2
1210,686176_2023-12-07,686176,E,2002,E2


In [10]:
df_min_distance = pd.read_sql('min_distance_to_region', con=ENGINE)
areas_of_interest = df_min_distance.columns[8:].to_list()

In [11]:
units = pq.ParquetDataset(npc_lims.get_cache_path('units'))
units.schema

num_spikes: double
firing_rate: double
presence_ratio: double
snr: double
isi_violations_ratio: double
isi_violations_count: double
rp_contamination: double
rp_violations: double
sliding_rp_violation: double
amplitude_cutoff: double
drift_ptp: double
drift_std: double
drift_mad: double
isolation_distance: double
l_ratio: double
d_prime: double
peak_to_valley: double
peak_trough_ratio: double
half_width: double
repolarization_slope: double
recovery_slope: double
electrode_group_name: string
peak_channel: int64
cluster_id: int64
default_qc: bool
amplitude: double
channels: list<element: int64>
  child 0, element: int64
unit_id: string
group_name: string
ccf_ap: double
ccf_dv: double
ccf_ml: double
structure: string
location: string
peak_electrode: int64
spike_times: list<element: double>
  child 0, element: double
obs_intervals: list<element: list<element: double>>
  child 0, element: list<element: double>
      child 0, element: double
electrodes: list<element: int64>
  child 0, element

In [12]:
s3 = s3fs.S3FileSystem()
electrode_groups_cache_dir = npc_lims.get_all_cache_paths(nwb_component='electrode_groups', version=VERSION)[0].parent.as_posix()[5:]
electrodes_cache_dir = npc_lims.get_all_cache_paths(nwb_component='electrodes', version=VERSION)[0].parent.as_posix()[5:]
units_cache_dir = npc_lims.get_all_cache_paths(nwb_component='units', version=VERSION)[0].parent.as_posix()[5:]

electrode_groups = pq.ParquetDataset(electrode_groups_cache_dir, filesystem=s3).read_pandas().to_pandas()
electrodes_original = pq.ParquetDataset(electrodes_cache_dir, filesystem=s3).read_pandas().to_pandas()
units_structures = pq.ParquetDataset(units_cache_dir, filesystem=s3).read(columns=['structure', 'location', 'peak_channel', 'subject_id', 'session_idx', 'date', 
                                      'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc']).to_pandas()

In [13]:
electrodes_original = dr.add_bool_columns(electrodes_original, version=VERSION)
units_structures_with_bools = dr.add_bool_columns(units_structures, version=VERSION)
units_structures_with_bools.dropna(inplace=True)
units_structures_with_bools.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc', 'session_id',
       'is_ephys', 'is_templeton', 'is_training', 'is_dynamic_routing',
       'is_opto'],
      dtype='object')

In [14]:
electrodes_original.columns

Index(['location', 'group_name', 'structure', 'x', 'y', 'z', 'channel',
       'rel_x', 'rel_y', 'reference', 'imp', 'session_idx', 'date',
       'subject_id', 'session_id', 'is_ephys', 'is_templeton', 'is_training',
       'is_dynamic_routing', 'is_opto'],
      dtype='object')

In [15]:
import ipywidgets

electrodes = electrodes_original.dropna()
electrodes = electrodes[~(electrodes['session_id'].str.contains('662892'))]
electrodes = electrodes[~(electrodes['structure'].str.islower())]
electrodes = electrodes[electrodes['is_dynamic_routing'] == True]

In [16]:
units_structures_with_bools_dr = units_structures_with_bools[units_structures_with_bools['is_dynamic_routing'] == True]
units_structures_with_bools_dr.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc', 'session_id',
       'is_ephys', 'is_templeton', 'is_training', 'is_dynamic_routing',
       'is_opto'],
      dtype='object')

In [17]:
from ipywidgets import interact, Box

In [18]:
def get_subject_colors(electrodes_structures_area: pd.DataFrame) -> list[tuple[int, int, int]]: 
    subject_colors = dr.utils.generate_subject_random_colors(electrodes_structures_area)
    subject_color_list = []

    for index, row in electrodes_structures_area.iterrows():
        subject_color_list.append(subject_colors[str(row['subject_id'])])
    
    return subject_color_list

In [19]:
unique_implants = df_ccf['Implant'].unique().tolist()
unique_implants.append('All')
implants_dropdown = ipywidgets.Dropdown(options=unique_implants, value=unique_implants[-1], description='Implants', disabled=False)

unique_holes = df_ccf['Hole'].unique().tolist()
unique_holes.append('All')
holes_dropdown = ipywidgets.Dropdown(options=unique_holes, value=unique_holes[-1], description='Holes', disabled=False)

unique_probes = df_ccf['Probe'].unique().tolist()
unique_probes.append('All')
probes_dropdown = ipywidgets.Dropdown(options=unique_probes, value=unique_probes[-1], description='Probes', disabled=False)

In [30]:
def update_3d_viewer_based_on_trajectory() -> None:
    urchin.ccf25.clear()
    urchin.clear_meshes()

    if implants_dropdown.value == 'All':
        pmeshes = urchin.meshes.create(len(electrodes)) #creates 2 primitives, stored in list pmeshes
        electrodes_sessions = electrodes
        coords = electrodes[['x', 'z', 'y']].to_numpy().tolist()
    else:
        probe_target_sessions = df_ccf[(df_ccf['Implant'] == implants_dropdown.value) & (df_ccf['Hole'] == holes_dropdown.value)
                                    & (df_ccf['Probe'] == probes_dropdown.value)]['session'].tolist()
        probe_target_sessions_filtered = []
        for session in probe_target_sessions:
            try:
                session_record = npc_session.SessionRecord(session)
                probe_target_sessions_filtered.append(f"{session_record.subject}_{session_record.date}_{session_record.idx}")
            except ValueError:
                pass
        
        electrodes_sessions = electrodes[electrodes['session_id'].isin(probe_target_sessions_filtered)]
        electrodes_sessions = electrodes_sessions[electrodes_sessions['group_name'] == f"probe{probes_dropdown.value}"]
        pmeshes = urchin.meshes.create(len(electrodes_sessions)) #creates 2 primitives, stored in list pmeshes
        coords = electrodes_sessions[['x', 'z', 'y']].to_numpy().tolist()
   
    
    print(f"Implant {implants_dropdown.value} Hole {holes_dropdown.value} Probe {probes_dropdown.value}")
    print(f"Sessions: {electrodes_sessions['session_id'].unique().tolist()}")
    print(f"Areas: {electrodes_sessions['structure'].unique().tolist()}")
    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    session_list = electrodes_sessions['session_id'].tolist()
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_sessions))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas(electrodes_sessions['structure'].unique().tolist())
    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")

In [21]:
def filter_by_implant(implant: str):
    if implant == 'All':
        holes_dropdown.options = unique_holes
        holes_dropdown.value = 'All'
    else:
        df_implant = df_ccf[df_ccf['Implant'] == implant]
        holes_dropdown.options = df_implant['Hole'].unique().tolist()
        holes_dropdown.value = holes_dropdown.options[0]

In [22]:
def filter_by_implant_hole(implant: str, hole: str):
    if implant == 'All' and hole == 'All':
        probes_dropdown.options = unique_probes
        probes_dropdown.value = 'All'
    else:
        df_implant_hole = df_ccf[(df_ccf['Implant'] == implant) & (df_ccf['Hole'] == hole)]
        probes_dropdown.options = sorted(df_implant_hole['Probe'].unique().tolist())
        probes_dropdown.value = probes_dropdown.options[0]

In [23]:
urchin.setup()

(URN) connected to server
Login sent with ID: 6ff1b22e, copy this ID into the renderer to connect.


In [24]:
urchin.ccf25.load()

In [25]:
@interact(x=['coronal', 'axial', 'sagittal'])
def camera_rotation(x):
    urchin.camera.main.set_rotation(x)
    urchin.camera.main.set_mode('perspective')
    urchin.camera.main.set_background_color('#ffffff')

interactive(children=(Dropdown(description='x', options=('coronal', 'axial', 'sagittal'), value='coronal'), Ou…

In [26]:
@interact(x=(5, 60))
def zoom(x=45):
    urchin.camera.main.set_zoom(60-x)

interactive(children=(IntSlider(value=45, description='x', max=60, min=5), Output()), _dom_classes=('widget-in…

In [27]:
items = [implants_dropdown, holes_dropdown, probes_dropdown]
box = Box(children=items)
box

Box(children=(Dropdown(description='Implants', index=8, options=('41', 'football', 'TS1', 'TS5', '2002', 'temp…

In [28]:
filter_by_implant(implants_dropdown.value)

In [34]:
filter_by_implant_hole(implants_dropdown.value, holes_dropdown.value)

In [35]:
update_3d_viewer_based_on_trajectory()

Implant 2002 Hole F1 Probe A
Sessions: ['664851_2023-11-13_0', '664851_2023-11-14_0', '668755_2023-08-30_0', '668755_2023-08-31_0', '676909_2023-12-12_0', '686740_2023-10-23_0', '686740_2023-10-24_0']
Areas: ['CP', 'SSp', 'PIR', 'OLF', 'CTXsp', 'EPd', 'AId', 'GU', 'MOp']


'2254'

In [28]:
unique_areas = electrodes['structure'].unique()
unique_areas.sort()
area_dropdown = ipywidgets.Dropdown(options=unique_areas, value=unique_areas[0], description='CCF Structure Area', disabled=False)
area_dropdown

@interact(x=unique_areas)
def update_3d_viewer(x):
    urchin.ccf25.clear()
    urchin.clear_meshes()
    units_structures_with_bools_dr_area = units_structures_with_bools_dr[units_structures_with_bools_dr['structure'] == x]
    electrodes_structures_area = electrodes[electrodes['structure'] == x]

    print(f"{x}: {len(electrodes_structures_area['subject_id'].unique())} mice")
    print(f"{x}: {len(electrodes_structures_area['session_id'].unique())} sessions")
    print(f"{x}: {len(units_structures_with_bools_dr_area)} units")

    pmeshes = urchin.meshes.create(len(electrodes_structures_area)) #creates 2 primitives, stored in list pmeshes
    coords = electrodes_structures_area[['x', 'z', 'y']].to_numpy().tolist()


    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_structures_area))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas([x])

    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")


interactive(children=(Dropdown(description='x', options=('ACAd', 'ACAv', 'ACB', 'AD', 'AId', 'AIv', 'AMd', 'AM…

In [33]:
area_dropdown = ipywidgets.Dropdown(options=areas_of_interest, value=areas_of_interest[0], description='Areas for min distance viewer', disabled=False)
area_dropdown

Dropdown(description='Areas for min distance viewer', options=('ACAd', 'ACAv', 'ACB', 'AD', 'AM', 'AON', 'APN'…

In [36]:
@interact(min_distance_to_area=(250, 500))
def min_distance_viewer(min_distance_to_area=300):
    print()

interactive(children=(IntSlider(value=300, description='min_distance_to_area', max=500, min=250), Output()), _…

In [ ]:
counts = {'area': [], 'num_units': [], 'num_sessions': [], 'num_subjects': []}
units_structure_areas_no_white_matter = units_structures_with_bools_dr[~(units_structures_with_bools_dr['structure'].str.islower())]
for area in units_structure_areas_no_white_matter['structure'].unique():
    units_structure_areas = units_structure_areas_no_white_matter[units_structure_areas_no_white_matter['structure'] == area]
    counts['area'].append(area)
    counts['num_sessions'].append(len(units_structure_areas['session_id'].unique()))
    counts['num_subjects'].append(len(units_structure_areas['subject_id'].unique()))
    counts['num_units'].append(len(units_structure_areas))

df_counts = pd.DataFrame(counts)

In [ ]:
df_counts.style.applymap(lambda x: 'background-color : red' if x>=3 else '', subset=['num_subjects']).applymap(lambda x: 'background-color : red' if x>150 else '', subset=['num_units'])

In [36]:
from IPython.display import display, Image, clear_output

def print_index(index):
    display(index)

urchin.meshes.callback = print_index

'2467'